# Class based AutoEncoder, Flow From Dataframe, Retraining Model, Call Backs

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   #https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from glob import glob
import numpy as np
import sys  
sys.path.insert(0, '/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/')
import tensorflow as tf
from models.AE import Autoencoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import pandas as pd

In [2]:
tf.__version__

'2.1.0'

In [3]:
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/'

TEST_FOLDER= RUN_FOLDER+ '/Testing/'
LOG_FOLDER= RUN_FOLDER+ '/logs'
if not os.path.exists(TEST_FOLDER):
    os.mkdir(TEST_FOLDER)
if not os.path.exists(LOG_FOLDER):
    os.mkdir(LOG_FOLDER)
    
print(RUN_FOLDER)

/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/


In [12]:
image_dataframe=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/image_location_venky.csv')
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32
NUM_IMAGES = 181858

In [5]:
def gray(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = np.expand_dims(gray_image, axis=-1)
    #print(gray_image.shape)
    return gray_image

In [14]:
data_gen = ImageDataGenerator(rescale=1./255, preprocessing_function=gray)

data_flow_dataframe = data_gen.flow_from_dataframe(dataframe = image_dataframe
                                         , directory = None
                                         , x_col='Path'
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = False
                                         , class_mode = 'input'
                                         , color_mode = 'rgb'
                                         #, save_to_dir = RUN_FOLDER
                                            )

Found 181858 validated image filenames.


In [7]:
ae = Autoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,48,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,48,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

ae.save(RUN_FOLDER)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [8]:
LEARNING_RATE = 0.0005
#R_LOSS_FACTOR = 10000
EPOCHS = 100
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [9]:
ae.compile(LEARNING_RATE)

In [10]:
for layer in ae.decoder.layers:
    #if(layer.get_config()['name']=='Activation'):
        print(layer.get_config())
        print(layer.__class__.__name__)
        print(type(layer.get_weights()))

{'batch_input_shape': (None, 200), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'decoder_input'}
InputLayer
<class 'list'>
{'name': 'dense', 'trainable': True, 'dtype': {'class_name': 'Policy', 'config': {'name': 'mixed_float16'}}, 'units': 4096, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
Dense
<class 'list'>
{'name': 'reshape', 'trainable': True, 'dtype': {'class_name': 'Policy', 'config': {'name': 'mixed_float16'}}, 'target_shape': (8, 8, 64)}
Reshape
<class 'list'>
{'name': 'decoder_conv_t_0', 'trainable': True, 'dtype': {'class_name': 'Policy', 'config': {'name': 'mixed_float16'}}, 'filters': 64, 'kernel_size': (3, 3), 'strides': (2, 2), 'padding': 'same', 'data_format': 'channels_last', 'dilati

In [15]:
ae.train_with_generator(     
    data_flow_dataframe
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

  ...
    to  
  ['...']
Train for 5683.0625 steps
Epoch 1/100
5682/5683 [============================>.] - ETA: 0s - loss: 0.0196
Epoch 00001: loss improved from inf to 0.01961, saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/model.h5

Epoch 00001: saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/weights.h5
5684/5683 [==============================] - 263s 46ms/step - loss: 0.0196
Epoch 2/100
5683/5683 [============================>.] - ETA: 0s - loss: 0.0178
Epoch 00002: loss improved from 0.01961 to 0.01776, saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/model.h5

Epoch 00002: saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/weights.h5
5684/5683 [==============================] - 262s 46ms/step - loss: 0.0178
Epoch 3/100
5682/5683 [============================>.] - ETA: 0s - loss: 0.

KeyboardInterrupt: 

In [24]:
!nvidia-smi -L

GPU 0: Tesla P40 (UUID: GPU-532949dc-9b98-a865-e163-da1dce6470d7)


# Retraining The Model

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras import layers
import os
#R_LOSS_FACTOR = 10000
import numpy as np
import cv2

In [3]:
# Custom loss function
def r_loss(y_true, y_pred):
            return K.mean(K.square(y_true - y_pred), axis = [1,2,3])

In [4]:


RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/'

TEST_FOLDER= RUN_FOLDER+ '/Testing/'
LOG_FOLDER= RUN_FOLDER+ '/logs'
if not os.path.exists(TEST_FOLDER):
    os.mkdir(TEST_FOLDER)
if not os.path.exists(LOG_FOLDER):
    os.mkdir(LOG_FOLDER)
    
print(RUN_FOLDER)

/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-500epochs/


In [5]:
# loading the saved model using keras load_model library
new_ae = load_model("/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/grey/AE-300epochs/model.h5", \
                     custom_objects={'r_loss':r_loss})
                    

In [6]:
LEARNING_RATE = 0.0005
#R_LOSS_FACTOR = 10000
BATCH_SIZE =1000
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [7]:
def gray(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = np.expand_dims(gray_image, axis=-1)
    #print(gray_image.shape)
    return gray_image

In [9]:
import pandas as pd
image_dataframe=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/image_location_venky.csv')
INPUT_DIM = (128,128,3)
BATCH_SIZE = 64
NUM_IMAGES = 181858

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen = ImageDataGenerator(rescale=1./255 , preprocessing_function=gray)
data_flow_dataframe = data_gen.flow_from_dataframe(dataframe = image_dataframe
                                         , directory = None
                                         , x_col='Path'
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = False
                                         , class_mode = 'input'
                                            )

Found 181858 validated image filenames.


In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping
import os
from datetime import datetime
logdir = os.path.join(RUN_FOLDER, "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logdir)

#checkpoint_filepath=os.path.join(run_folder, "model.h5")
checkpoint1 = ModelCheckpoint(os.path.join(RUN_FOLDER, "model.h5"), monitor='loss',  save_best_only=True, verbose=1, mode= 'min')
checkpoint2 = ModelCheckpoint(os.path.join(RUN_FOLDER, 'weights.h5'), save_weights_only = True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, mode='min', verbose=1)
EarlyStop = EarlyStopping(monitor='loss', min_delta=0.0001, patience=31, verbose=1, mode='min', restore_best_weights=True)


callbacks_list = [checkpoint1, checkpoint2, reduce_lr, tensorboard_callback, EarlyStop]

In [10]:
new_ae.fit_generator(
            data_flow_dataframe
            , shuffle = True
            , epochs = EPOCHS
            , initial_epoch = INITIAL_EPOCH
            , callbacks = callbacks_list
            , steps_per_epoch=NUM_IMAGES / BATCH_SIZE
            , workers= 7
            , use_multiprocessing=False
            )



Epoch 1/200
 151/2841 [>.............................] - ETA: 3:57 - loss: 0.0136

/opt/conda/lib/python3.6/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 629/2841 [=====>........................] - ETA: 2:56 - loss: 0.0140

KeyboardInterrupt: 